In [3]:
!pip install cmake
!pip install dlib


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import cv2  # Biblioteca para el procesamiento de imágenes en tiempo real
import face_recognition  # Biblioteca para el reconocimiento facial
import os  

In [2]:
# Directorio donde se almacenan las imágenes de los rostros
rostros_dir = 'rostros/'

# Listas para almacenar las codificaciones de los rostros y sus nombres
known_face_encodings = []
known_face_names = []

# Recorrer el directorio y procesar cada imagen
for filename in os.listdir(rostros_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Cargar la imagen
        image_path = os.path.join(rostros_dir, filename)
        image = face_recognition.load_image_file(image_path)
        
        # Codificar el rostro
        face_encoding = face_recognition.face_encodings(image)[0]
        
        # Agregar la codificación y el nombre a las listas
        known_face_encodings.append(face_encoding)
        known_face_names.append(os.path.splitext(filename)[0])  # Usar el nombre del archivo sin la extensión como nombre del rostro


In [4]:
# Iniciar la captura de video desde la cámara
video_capture = cv2.VideoCapture(0)

while True:
    # Capturar un solo cuadro de video
    ret, frame = video_capture.read()
    
    # Reducir el tamaño del cuadro para acelerar el procesamiento
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    
    # Convertir el cuadro de BGR a RGB
    rgb_small_frame = small_frame[:, :, ::-1]
    
    # Encontrar todos los rostros y sus codificaciones en el cuadro actual
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
    
    # Inicializar una lista para los nombres de los rostros reconocidos
    face_names = []
    
    # Comparar cada codificación de rostro con las codificaciones conocidas
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"
        
        # Utilizar el rostro conocido con la menor distancia de codificación
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = face_distances.argmin()
        if matches[best_match_index]:
            name = known_face_names[best_match_index]
        
        face_names.append(name)
    
    # Mostrar los resultados
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Escalar las ubicaciones del rostro a partir del tamaño reducido
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
        
        # Dibujar un cuadro alrededor del rostro
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        
        # Dibujar una etiqueta con el nombre debajo del rostro
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
    
    # Mostrar la imagen resultante
    cv2.imshow('Video', frame)
    
    # Salir del bucle cuando se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar el manejador de la cámara y cerrar las ventanas
video_capture.release()
cv2.destroyAllWindows()


[ WARN:0@34.847] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@34.847] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
